In [161]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from copy import deepcopy

from sklearn.preprocessing import MinMaxScaler

In [162]:
import pandas as pd

# Load the results and races datasets
results = pd.read_csv('dataset/results.csv')
races = pd.read_csv('dataset/races.csv')
races['date'] = pd.to_datetime(races['date'])
# sort by date
races = races.sort_values('date')
races['sorted_id'] = races.reset_index(drop=True).index + 1
drivers = pd.read_csv('dataset/drivers.csv')

# save races as races_sorted csv
races.to_csv('races_time_sorted.csv', index=False)

In [163]:
# join them results on drivers using driverrId

results= results.merge(drivers, on='driverId')
# join on race using raceId

results = results.merge(races, on='raceId')

In [164]:
results.head()

,resultId,raceId,driverId,constructorId,number_x,grid,position,positionText,positionOrder,points,...,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,sorted_id
0,1,18,1,1,22,1,1,1,1,10.0,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,786
1,2,18,2,2,3,5,2,2,2,8.0,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,786
2,3,18,3,3,7,7,3,3,3,6.0,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,786
3,4,18,4,4,5,11,4,4,4,5.0,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,786
4,5,18,5,1,23,3,5,5,5,4.0,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,786


In [165]:
# save to temp_results.csv
results.to_csv('temp_results.csv', index=False)

In [166]:
# choose the columns we want we onlt want reusltId and raceId
results_cpy = results[['resultId' ,'raceId','driverId','position','driverRef','year','sorted_id']]
# sort on sorted_id
results_cpy = results_cpy.sort_values('sorted_id')

In [167]:
results_cpy.head()

,resultId,raceId,driverId,position,driverRef,year,sorted_id
25810,20044,833,661,\N,peter_walker,1950,1
25807,20041,833,640,\N,graffenried,1950,1
25811,20178,833,666,\N,rolt,1950,1
25812,20038,833,669,\N,bira,1950,1
25813,20027,833,686,3,reg_parnell,1950,1


In [168]:
# drop where postion in \\N
results_cpy = results_cpy[results_cpy['position'] != '\\N']
# make it int
results_cpy['position'] = results_cpy['position'].astype(int)

#only use reults from 1995 to 2022
results_cpy = results_cpy[results_cpy['year'] >= 1960]
results_cpy = results_cpy[results_cpy['year'] <= 2022]

In [169]:
# want to make a frame having all drivers exwept with driverid ==1 
# and another frame having only dirverid ==1

# get all drivers
# make reults cpy have only positon between 1 and 20
results_cpy = results_cpy[results_cpy['position'] <= 20]
results_cpy = results_cpy[results_cpy['position'] >= 1]
results_without_ham = results_cpy[results_cpy['driverId'] != 1]
# sort on raceId
results_ham = results_without_ham.sort_values(by=['raceId'])


# get only hamilton
results_ham = results_cpy[results_cpy['driverId'] == 1]
# sort on raceId
results_ham = results_ham.sort_values(by=['raceId'])

In [170]:
# make each driver have a list of all his races
# make a list of all drivers
drivers = results_without_ham['driverId'].unique()


In [171]:
results_ham.head()

,resultId,raceId,driverId,position,driverRef,year,sorted_id
762,7580,2,1,7,hamilton,2009,805
782,7599,3,1,6,hamilton,2009,806
802,7617,4,1,4,hamilton,2009,807
822,7642,5,1,9,hamilton,2009,808
842,7665,6,1,12,hamilton,2009,809


In [172]:
# sort on sotred_id
results_ham = results_ham.sort_values(by=['sorted_id'])
results_without_ham.head()

,resultId,raceId,driverId,position,driverRef,year,sorted_id
23604,18077,746,499,11,bonomi,1960,85
23605,18079,746,500,13,munaron,1960,85
23592,18072,746,404,6,ireland,1960,85
23603,18076,746,498,10,gonzalez,1960,85
23602,18075,746,497,9,larreta,1960,85


In [173]:
# for each driver
# store his info


# make a list of all drivers
drivers = results_without_ham['driverId'].unique()
drivers.sort()
print(drivers)

# make a dict of all drivers
drivers_dict = {}
for driver in drivers:
    drivers_dict[driver] = []


[  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  29  30  31  32  33  34  35  36  37  38
  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56
  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  73  74  75
  76  77  78  79  81  82  83  84  85  86  87  88  89  90  91  92  93  94
  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112
 113 114 116 117 118 119 120 121 122 123 125 126 127 128 129 131 133 136
 137 138 139 140 141 145 146 147 148 151 152 153 154 155 156 157 158 159
 160 163 165 166 167 168 169 170 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 192 193 194 195 196 197 199 200 201
 202 203 205 206 207 208 209 211 212 216 217 219 221 222 223 224 229 230
 231 232 233 235 236 237 238 239 240 243 246 249 250 251 252 253 254 255
 256 258 259 262 265 267 269 270 271 272 273 274 275 276 277 278 280 282
 284 286 287 288 289 290 291 292 293 294 296 297 29

In [174]:
# loop over all drivers
sorted_keys = sorted(drivers_dict.keys())
for id_key in sorted_keys:
    wanted_df = results_without_ham[results_without_ham['driverId'] == id_key]
    for index, row in wanted_df.iterrows():
        if row['driverId'] == id_key:
            drivers_dict[id_key].append(row['position'])

In [175]:
# print count of those who have more than 120 elemnt
count =0
for key in drivers_dict:
    if len(drivers_dict[key]) > 30:
        count +=1
        # print(key, len(drivers_dict[key]))
print(count)


129


In [176]:
new_drivers_dict = deepcopy(drivers_dict)
for key in new_drivers_dict:
    while len(new_drivers_dict[key]) < 30 and len(new_drivers_dict[key]) > 20:
        # new_drivers_dict[key].append(0)
        numbers = deepcopy(new_drivers_dict[key])
        indices = np.arange(len(numbers))

        # Interpolate to a length of 30 using the unsorted indices
        interp_indices = np.linspace(0, len(numbers)-1, 30)
        interp_numbers = np.interp(interp_indices, indices, numbers)

        # Round the interpolated values to integers
        interp_numbers = np.round(interp_numbers).astype(int)
        new_drivers_dict[key] = interp_numbers


# make all of them at max 30
for key in new_drivers_dict:
    new_drivers_dict[key] = new_drivers_dict[key][:30]

# print len of them all
for key in new_drivers_dict:
    print(key, len(new_drivers_dict[key]))
# remove anything having less than 30
key_to_remove = []
for key in new_drivers_dict:
    if len(new_drivers_dict[key]) < 30:
        key_to_remove.append(key)
for key in key_to_remove:
    del new_drivers_dict[key]
# assert that all have 30
for key in new_drivers_dict:
    assert len(new_drivers_dict[key]) == 30

2 30
3 30
4 30
5 30
6 30
7 20
8 30
9 30
10 30
11 30
12 17
13 30
14 30
15 30
16 30
17 30
18 30
19 14
20 30
21 30
22 30
23 30
24 30
25 30
26 17
27 30
29 14
30 30
31 30
32 30
33 30
34 1
35 30
36 3
37 30
38 9
39 30
40 5
41 19
42 11
43 20
44 30
45 6
46 9
47 12
48 30
49 30
50 30
51 7
52 8
53 5
54 9
55 30
56 30
57 30
58 10
59 9
60 13
61 2
62 6
63 30
64 30
65 30
66 8
67 30
68 12
69 30
70 18
71 30
73 12
74 4
75 18
76 9
77 30
78 30
79 30
81 30
82 3
83 19
84 30
85 4
86 2
87 30
88 30
89 5
90 17
91 18
92 15
93 3
94 30
95 30
96 7
97 2
98 1
99 13
100 30
101 7
102 30
103 30
104 30
105 30
106 4
107 1
108 5
109 30
110 30
111 4
112 30
113 1
114 15
116 1
117 30
118 30
119 30
120 3
121 6
122 30
123 30
125 3
126 2
127 30
128 3
129 14
131 30
133 30
136 12
137 30
138 30
139 17
140 30
141 5
145 30
146 3
147 4
148 2
151 6
152 30
153 30
154 30
155 30
156 30
157 15
158 30
159 11
160 20
163 30
165 2
166 30
167 2
168 1
169 8
170 18
172 30
173 30
174 6
175 30
176 30
177 30
178 30
179 6
180 3
181 13
182 30
183 5
184 

In [177]:
len(new_drivers_dict)

163

In [178]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Prepare the data
# data = {'driver1': [1, 2, 3, ..., 30], 'driver2': [2, 3, 4, ..., 1], ...}
key_sorted = sorted(new_drivers_dict.keys())
X= []
y= []
for key in key_sorted:
    X.append(new_drivers_dict[key][:25])
    y.append(new_drivers_dict[key][25:])


In [179]:
# train test split size = 0.8
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)


In [180]:
class RacePredictor(nn.Module):
    def __init__(self):
        super(RacePredictor, self).__init__()
        self.fc1 = nn.Linear(25, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = torch.sigmoid(x) * 19 + 1
        return x

In [181]:

# Instantiate the neural network and optimizer
model = RacePredictor()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
# criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 1000
batch_size = 32
num_batches = len(X_train) // batch_size

for epoch in range(num_epochs):
    epoch_loss = 0.0
    for i in range(num_batches):
        X_batch = torch.tensor(X_train[i*batch_size:(i+1)*batch_size], dtype=torch.float32)
        y_batch = torch.tensor(y_train[i*batch_size:(i+1)*batch_size], dtype=torch.float32)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    if num_epochs >0 and num_batches > 0    :
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss / num_batches}')


Epoch 1/1000, Loss: 32.74926996231079
Epoch 2/1000, Loss: 24.91571283340454
Epoch 3/1000, Loss: 22.757938861846924
Epoch 4/1000, Loss: 22.092408180236816
Epoch 5/1000, Loss: 21.36563539505005
Epoch 6/1000, Loss: 20.726301193237305
Epoch 7/1000, Loss: 20.228209495544434
Epoch 8/1000, Loss: 19.587996244430542
Epoch 9/1000, Loss: 18.887115240097046
Epoch 10/1000, Loss: 18.256640911102295
Epoch 11/1000, Loss: 17.72216272354126
Epoch 12/1000, Loss: 17.061705827713013
Epoch 13/1000, Loss: 16.402437210083008
Epoch 14/1000, Loss: 15.811817646026611
Epoch 15/1000, Loss: 15.237550735473633
Epoch 16/1000, Loss: 14.69635272026062
Epoch 17/1000, Loss: 14.177331924438477
Epoch 18/1000, Loss: 13.692246675491333
Epoch 19/1000, Loss: 13.249711513519287
Epoch 20/1000, Loss: 12.88208293914795
Epoch 21/1000, Loss: 12.494439363479614
Epoch 22/1000, Loss: 12.158023834228516
Epoch 23/1000, Loss: 11.837816953659058
Epoch 24/1000, Loss: 11.561240196228027
Epoch 25/1000, Loss: 11.250359535217285
Epoch 26/1000, 

In [182]:

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [183]:

with torch.no_grad():
    output = model(X_test)
    mse = criterion(output, y_test)
print(f'Test MSE: {mse.item()}')


Test MSE: 38.05239486694336


In [184]:

# Make predictions
with torch.no_grad():
    predictions = model(X_test)
    predictions = predictions.numpy()
predictions = np.round(predictions)


In [185]:
print(predictions)

[[ 3.  2.  2.  3.  4.]
 [17.  8. 12. 16. 17.]
 [ 8.  6. 11.  9. 15.]
 [13. 15. 14. 17. 17.]
 [11.  9. 16.  8. 14.]
 [ 5. 13. 14.  7. 12.]
 [ 5.  2.  7.  3.  4.]
 [ 3.  4.  4. 11.  8.]
 [ 2.  3.  6.  1.  5.]
 [ 2.  3.  9.  8. 15.]
 [ 2.  3.  2.  3.  3.]
 [ 4.  3.  1.  1.  6.]
 [12.  9. 13. 18. 14.]
 [13. 14.  4.  2.  9.]
 [14. 10. 18.  9. 13.]
 [11.  4.  8.  4. 16.]
 [ 7. 13. 13. 13. 17.]]


In [186]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# print acc
count = 0
for i in range(len(predictions)):
    np_pred = np.array(predictions[i]) 
    np_y_test = np.array(y_test[i]) 

    similarity = cosine_similarity([np_pred, np_y_test])
    cosine_sim = similarity[0][1]
    print(np_pred )
    print(np_y_test )
    print(cosine_sim)
    # print abs diff
    diff = np.abs(np_pred - np_y_test)
    print(diff)
    print('===================================================================================')
    # print(cosine_sim)

   


[3. 2. 2. 3. 4.]
[11.  7.  6.  2.  2.]
0.77000105
[8. 5. 4. 1. 2.]
[17.  8. 12. 16. 17.]
[17. 17. 13. 19. 15.]
0.96728826
[0. 9. 1. 3. 2.]
[ 8.  6. 11.  9. 15.]
[17.  6. 16.  6.  8.]
0.871348
[9. 0. 5. 3. 7.]
[13. 15. 14. 17. 17.]
[13.  9.  9. 14. 12.]
0.9849945
[0. 6. 5. 3. 5.]
[11.  9. 16.  8. 14.]
[17. 15. 14. 14. 14.]
0.96007425
[6. 6. 2. 6. 0.]
[ 5. 13. 14.  7. 12.]
[15. 12. 12. 16. 15.]
0.9077172
[10.  1.  2.  9.  3.]
[5. 2. 7. 3. 4.]
[ 8. 12. 10. 11.  6.]
0.8727469
[ 3. 10.  3.  8.  2.]
[ 3.  4.  4. 11.  8.]
[17. 13.  9. 14. 15.]
0.8866663
[14.  9.  5.  3.  7.]
[2. 3. 6. 1. 5.]
[17. 16. 16. 11. 10.]
0.8632603
[15. 13. 10. 10.  5.]
[ 2.  3.  9.  8. 15.]
[ 5.  8. 10.  7.  4.]
0.76947594
[ 3.  5.  1.  1. 11.]
[2. 3. 2. 3. 3.]
[1. 1. 2. 3. 6.]
0.85208595
[1. 2. 0. 0. 3.]
[4. 3. 1. 1. 6.]
[ 6. 15. 13.  8.  8.]
0.73602355
[ 2. 12. 12.  7.  2.]
[12.  9. 13. 18. 14.]
[11. 18. 14. 16. 12.]
0.95547086
[1. 9. 1. 2. 2.]
[13. 14.  4.  2.  9.]
[ 8.  6. 15.  3. 10.]
0.76492816
[ 5.  8. 11.  1.

#### INsights from what happened
### 1. The model is not good at predicting the trend of the time series
### 2. As the racers performance does not follow a trend, it is hard to predict the performance of the racer
### 3. Each driver has a different performance, so it is hard to predict the performance of all drivers using only one model
### 4. We can fine tune the model using the data of a single driver to predict the performance of that driver
### 5. So for each driver, we should fine tune the model for him to get a good prediction

In [188]:
#divide results_ham to lists of 30
# sort results_ham

# sort on resultsId
df = results_ham.sort_values(by="resultId")

df.head()

results_ham_30 = []

for i in range(0, len(df), 30):
    results_ham_30.append(results_ham[i:i+30])

# make it lists of 30
# having the postion

result_list_of_30 = []
for i in range(len(results_ham_30)):
    temp = []
    for j in range(len(results_ham_30[i])):
        temp.append(results_ham_30[i].iloc[j]['position'])
    result_list_of_30.append(temp)


len(result_list_of_30[-1])

# delete last one
result_list_of_30 = result_list_of_30[:-3]
len(result_list_of_30[-1])

# assert all of 30
for i in range(len(result_list_of_30)):
    assert len(result_list_of_30[i]) == 30


In [189]:
print(len(result_list_of_30))
X_lewis = []
y_lewis = []
for i in range(len(result_list_of_30)):
    X_lewis.append(result_list_of_30[i][:25])
    y_lewis.append(result_list_of_30[i][25:])

# train is all expet hte last one
# test is the last one
X_lewis_train = X_lewis[:-1]
y_lewis_train = y_lewis[:-1]

X_lewis_test = X_lewis[-1]
y_lewis_test = y_lewis[-1]

7


In [190]:
batch_size = 1
num_batches = len(X_lewis_train) // batch_size
criterion = nn.MSELoss()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for i in range(num_batches):
        X_batch = torch.tensor(X_lewis_train[i*batch_size:(i+1)*batch_size], dtype=torch.float32)
        y_batch = torch.tensor(y_lewis_train[i*batch_size:(i+1)*batch_size], dtype=torch.float32)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    if num_epochs >0 and num_batches > 0    :
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss / num_batches}')

Epoch 1/1000, Loss: 17.50426471233368
Epoch 2/1000, Loss: 12.182545324166616
Epoch 3/1000, Loss: 4.654301623503367
Epoch 4/1000, Loss: 5.192656576633453
Epoch 5/1000, Loss: 4.383506307999293
Epoch 6/1000, Loss: 4.272652248541514
Epoch 7/1000, Loss: 3.5452629576126733
Epoch 8/1000, Loss: 3.842450112104416
Epoch 9/1000, Loss: 3.9486470917860665
Epoch 10/1000, Loss: 3.542758896946907
Epoch 11/1000, Loss: 3.7771021127700806
Epoch 12/1000, Loss: 3.6975914537906647
Epoch 13/1000, Loss: 3.4034284378091493
Epoch 14/1000, Loss: 3.3221184412638345
Epoch 15/1000, Loss: 2.712949345509211
Epoch 16/1000, Loss: 2.9715136041243873
Epoch 17/1000, Loss: 2.7440830767154694
Epoch 18/1000, Loss: 3.251510957876841
Epoch 19/1000, Loss: 3.1776841084162393
Epoch 20/1000, Loss: 3.325065235296885
Epoch 21/1000, Loss: 3.054335307329893
Epoch 22/1000, Loss: 2.6866148685415587
Epoch 23/1000, Loss: 2.3541068447132907
Epoch 24/1000, Loss: 2.262270920909941
Epoch 25/1000, Loss: 2.0387889351695776
Epoch 26/1000, Loss: 

In [191]:
X_lewis_test = torch.tensor(X_lewis_test, dtype=torch.float32)
y_lewis_test = torch.tensor(y_lewis_test, dtype=torch.float32)

In [192]:

with torch.no_grad():
    output = model(X_lewis_test)
    mse = criterion(output, y_lewis_test)
print(f'Test MSE: {mse.item()}')


Test MSE: 0.43985652923583984


In [196]:

# Make predictions
with torch.no_grad():
    predictions = model(X_lewis_test)
    predictions = predictions.numpy()
predictions = np.round(predictions)
print("predictes_positions:", predictions)
print("Ground truth:       ",y_lewis_test.numpy())

predictes_positions: [2. 1. 2. 2. 2.]
Ground truth:        [2. 1. 1. 2. 1.]


# As the data is sequential, let's try LSTM model

In [197]:
data = deepcopy(list(zip(X_train, y_train)))

In [198]:
import numpy as np

def one_hot_encode(data, num_classes):
    encoded_data = []
    for sublist in data:
        encoded_sublist = np.zeros((len(sublist), num_classes))
        encoded_sublist[np.arange(len(sublist)), np.array(sublist)-1] = 1
        encoded_data.append(encoded_sublist.tolist())
    return encoded_data


In [199]:

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

class LSTMRacePosition(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMRacePosition, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        out = F.softmax(out, dim=2)
        return out.transpose(1, 2)
# Define the dataset


In [344]:

class RaceData(Dataset):
    def __init__(self, X, y):
        # print(X)
        self.X = torch.tensor(one_hot_encode(X,20), dtype=torch.float32)
        self.y = torch.tensor(one_hot_encode(y,20), dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]



In [345]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

# # to numpy  
# X_train = np.array(X_train)
# X_test = np.array(X_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test)
print(len(y_train[0]))

10


In [346]:
dataset = RaceData(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)


In [347]:
model = LSTMRacePosition(input_size=20, hidden_size=64, output_size=10)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
temp = None
num_epochs = 2000
for epoch in range(num_epochs):
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        output = model(batch_x)
        # print(output.shape)
        # print(batch_y.shape)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        temp = output.shape
    print("Epoch: {}, Loss: {}".format(epoch+1, loss.item()))
print(temp)

Epoch: 1, Loss: 0.05002117156982422
Epoch: 2, Loss: 0.0500025600194931
Epoch: 3, Loss: 0.049992531538009644
Epoch: 4, Loss: 0.04998767748475075
Epoch: 5, Loss: 0.04998258128762245
Epoch: 6, Loss: 0.04997546598315239
Epoch: 7, Loss: 0.04996810853481293
Epoch: 8, Loss: 0.04996103793382645
Epoch: 9, Loss: 0.04995385557413101
Epoch: 10, Loss: 0.04994618892669678
Epoch: 11, Loss: 0.049937836825847626
Epoch: 12, Loss: 0.04992857947945595
Epoch: 13, Loss: 0.04991809278726578
Epoch: 14, Loss: 0.04990602284669876
Epoch: 15, Loss: 0.049891937524080276
Epoch: 16, Loss: 0.049875203520059586
Epoch: 17, Loss: 0.04985472187399864
Epoch: 18, Loss: 0.04982858523726463
Epoch: 19, Loss: 0.04979352653026581
Epoch: 20, Loss: 0.04974406212568283
Epoch: 21, Loss: 0.04967445135116577
Epoch: 22, Loss: 0.04959264397621155
Epoch: 23, Loss: 0.04950718209147453
Epoch: 24, Loss: 0.049401044845581055
Epoch: 25, Loss: 0.049266617745161057
Epoch: 26, Loss: 0.04912607744336128
Epoch: 27, Loss: 0.04902291297912598
Epoch

In [348]:
# get cosine similarity on test set
with torch.no_grad():
    predictions = model(torch.tensor(one_hot_encode(X_test,20), dtype=torch.float32))
    predictions = predictions.argmax(dim=2).tolist()

    # predictions = predictions.numpy()
# make only the max value 1 and the rest 0


In [349]:
print(predictions)
# add one to each value
for i in range(len(predictions)):
    for j in range(len(predictions[i])):
        predictions[i][j] += 1
print(predictions)



[[19, 4, 10, 6, 8, 13, 5, 4, 11, 1], [8, 6, 13, 3, 6, 14, 13, 11, 12, 10], [13, 14, 5, 10, 12, 3, 8, 7, 9, 2], [8, 1, 6, 0, 3, 11, 5, 7, 9, 10], [10, 14, 15, 16, 7, 2, 17, 11, 8, 12], [17, 7, 19, 5, 14, 4, 12, 6, 8, 5], [12, 1, 8, 10, 3, 7, 11, 13, 4, 14], [5, 13, 12, 1, 2, 7, 3, 9, 10, 4], [8, 17, 12, 11, 10, 13, 14, 6, 2, 15], [6, 5, 10, 3, 12, 9, 7, 1, 8, 10], [8, 4, 10, 13, 4, 5, 7, 11, 14, 0], [9, 2, 12, 6, 12, 4, 7, 1, 4, 5], [3, 17, 16, 12, 11, 14, 8, 1, 0, 10], [15, 3, 13, 9, 11, 12, 1, 5, 18, 8], [17, 11, 13, 7, 12, 14, 19, 8, 4, 10], [4, 7, 11, 6, 1, 10, 9, 8, 0, 3], [12, 13, 6, 7, 14, 8, 18, 19, 5, 11]]
[[20, 5, 11, 7, 9, 14, 6, 5, 12, 2], [9, 7, 14, 4, 7, 15, 14, 12, 13, 11], [14, 15, 6, 11, 13, 4, 9, 8, 10, 3], [9, 2, 7, 1, 4, 12, 6, 8, 10, 11], [11, 15, 16, 17, 8, 3, 18, 12, 9, 13], [18, 8, 20, 6, 15, 5, 13, 7, 9, 6], [13, 2, 9, 11, 4, 8, 12, 14, 5, 15], [6, 14, 13, 2, 3, 8, 4, 10, 11, 5], [9, 18, 13, 12, 11, 14, 15, 7, 3, 16], [7, 6, 11, 4, 13, 10, 8, 2, 9, 11], [9, 5, 1

In [350]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# print acc
count = 0
for i in range(len(predictions)):
    np_pred = np.array(predictions[i]) 
    np_y_test = np.array(y_test[i]) 

    similarity = cosine_similarity([np_pred, np_y_test])
    cosine_sim = similarity[0][1]
    print(np_pred )
    print(np_y_test )
    print(cosine_sim)
    print('===================================================================================')
    print(cosine_sim)

   

[20  5 11  7  9 14  6  5 12  2]
[ 8  2  2  5  3 11  7  6  2  2]
0.8637257676283169
0.8637257676283169
[ 9  7 14  4  7 15 14 12 13 11]
[15 18 12 13 15 17 17 13 19 15]
0.9484771988421224
0.9484771988421224
[14 15  6 11 13  4  9  8 10  3]
[ 8 14 15 14 11 17  6 16  6  8]
0.8492846143086356
0.8492846143086356
[ 9  2  7  1  4 12  6  8 10 11]
[13 11 11 11 12 13  9  9 14 12]
0.9100816894838395
0.9100816894838395
[11 15 16 17  8  3 18 12  9 13]
[14 17 15 16 12 17 15 14 14 14]
0.9404413704260469
0.9404413704260469
[18  8 20  6 15  5 13  7  9  6]
[16 12 15 11 13 15 12 12 16 15]
0.9109449586809105
0.9109449586809105
[13  2  9 11  4  8 12 14  5 15]
[10 10  6  6  5  8 12 10 11  6]
0.8774575823156006
0.8774575823156006
[ 6 14 13  2  3  8  4 10 11  5]
[11 17 17 14 12 17 13  9 14 15]
0.9011317615411529
0.9011317615411529
[ 9 18 13 12 11 14 15  7  3 16]
[16  9 14 17 14 17 16 16 11 10]
0.9090832095767554
0.9090832095767554
[ 7  6 11  4 13 10  8  2  9 11]
[13 11  8  6  6  5  8 10  7  4]
0.8173922982766013

#### The LSTM model did not work well as the data is not enough

## In Conclusion
1. The NN model is not good at predicting the trend of the time series
2. To have a better performance, you have to fine tune on a specefic driver to get a good prediction that is close enough to the real data
3. The LSTM model did not work well as the data is not enough, if the data was enough and we had more time to find a suitable loss function and tune the model, we could have got a better result